In [ ]:
# NFL Draft Prediction - 探索的データ分析（EDA）

## 概要
NFL選手のドラフト指名予測のための包括的なデータ分析を行います。

## 目標
- データの構造と品質を理解
- 欠損値パターンの分析
- 特徴量とターゲットの関係性の発見
- ドメイン知識の活用とインサイトの獲得


In [ ]:
## 1. セットアップ


In [ ]:
# 基本ライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats

# 設定
plt.style.use('default')
sns.set_palette("husl")
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# 日本語フォント設定（必要に応じて）
plt.rcParams['font.family'] = ['DejaVu Sans', 'Arial', 'sans-serif']

print("ライブラリの読み込み完了")


In [ ]:
## 2. データ読み込み


In [ ]:
# データ読み込み
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample_submission = pd.read_csv('data/sample_submission_6_20修正.csv')

print(f"訓練データサイズ: {train.shape}")
print(f"テストデータサイズ: {test.shape}")
print(f"サンプル提出ファイルサイズ: {sample_submission.shape}")

# データの最初の数行を確認
print("\n=== 訓練データ（先頭5行）===")
display(train.head())
print("\n=== テストデータ（先頭5行）===")
display(test.head())


In [ ]:
## 3. 基本統計量とデータ概要


In [ ]:
# データの基本情報
print("=== 訓練データの基本情報 ===")
print(train.info())
print("\n=== 訓練データの基本統計量 ===")
print(train.describe())

# カテゴリ変数の確認
print("\n=== カテゴリ変数の概要 ===")
categorical_cols = train.select_dtypes(include=['object']).columns.tolist()
for col in categorical_cols:
    print(f"\n{col}: {train[col].nunique()}個のユニーク値")
    print(train[col].value_counts().head())


In [ ]:
## 4. 目的変数（Drafted）の分析


In [ ]:
# 目的変数の分布分析
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# ヒストグラム
axes[0].hist(train['Drafted'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
axes[0].set_title('Drafted値の分布')
axes[0].set_xlabel('Drafted')
axes[0].set_ylabel('頻度')

# 二値化での円グラフ
drafted_binary = (train['Drafted'] > 0.5).astype(int)
drafted_counts = drafted_binary.value_counts()
axes[1].pie(drafted_counts.values, labels=['指名されず', '指名された'], autopct='%1.1f%%',
           colors=['lightcoral', 'lightgreen'])
axes[1].set_title('ドラフト指名の割合（二値化）')

plt.tight_layout()
plt.show()

print(f"Drafted値の平均: {train['Drafted'].mean():.4f}")
print(f"指名された選手の割合: {drafted_binary.mean():.4f}")
print(f"指名された選手数: {drafted_binary.sum()}人")
print(f"指名されなかった選手数: {len(drafted_binary) - drafted_binary.sum()}人")


In [ ]:
## 5. 欠損値の詳細分析


In [ ]:
# 欠損値の詳細分析
def analyze_missing_values(df, title):
    missing_data = df.isnull().sum()
    missing_percent = 100 * missing_data / len(df)
    
    missing_table = pd.DataFrame({
        '欠損数': missing_data,
        '欠損率(%)': missing_percent
    })
    missing_table = missing_table[missing_table['欠損数'] > 0].sort_values('欠損数', ascending=False)
    
    print(f"=== {title} 欠損値分析 ===")
    if len(missing_table) > 0:
        print(missing_table)
        
        # 可視化
        plt.figure(figsize=(10, 6))
        sns.barplot(data=missing_table.reset_index(), x='欠損率(%)', y='index')
        plt.title(f'{title} - 欠損率(%)')
        plt.tight_layout()
        plt.show()
    else:
        print("欠損値はありません")
    
    return missing_table

# 訓練データとテストデータの欠損値分析
train_missing = analyze_missing_values(train, "訓練データ")
test_missing = analyze_missing_values(test, "テストデータ")
